In [11]:
# link colab and drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [61]:
# CV project
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import os
import torchvision
from torchvision import datasets, transforms, models
from IPython import display
import shelve
from PIL import Image
import glob
import matplotlib
from matplotlib.pyplot import *

# if available use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

trainPath = '/content/drive/My Drive/Units/CV_project/Images/train'
testPath =  '/content/drive/My Drive/Units/CV_project/Images/test'
dataTrain = torchvision.datasets.ImageFolder(trainPath)
dataTest = torchvision.datasets.ImageFolder(testPath)
print(dataTrain.classes)
print(dataTrain.imgs[99][0])
print(len(dataTrain.imgs[99][0]))
im = Image.open(dataTrain.imgs[1][0])
%matplotlib inline
im
ii=torch.from_numpy(np.asarray(im))
ii

Device: cpu
['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']
/content/drive/My Drive/Units/CV_project/Images/train/Bedroom/image_0216.jpg
76


tensor([[208, 183, 181,  ..., 111, 111, 110],
        [207, 183, 181,  ..., 111, 111, 110],
        [208, 184, 181,  ..., 112, 112, 110],
        ...,
        [101, 101,  99,  ...,  28,  28,  29],
        [ 96,  93,  93,  ...,  24,  25,  26],
        [ 98,  96,  98,  ...,  25,  25,  24]], dtype=torch.uint8)

In [70]:
from torch.utils.data import Dataset, Sampler, DataLoader, SubsetRandomSampler

class customDataset(Dataset):
  '''
    Our dataset is a list of lists: [path_to_image, class]
  '''

  def __init__(self, root_path, transform):
    '''
      This function takes in input the path of the folder containing class folders,
      the tranformation to be applied to each image and creates the dataset
    '''
    self.data = torchvision.datasets.ImageFolder(root_path)
    
  def __getitem__(self, key):
    '''
      This function access the image, transforms it and returns: the image as a tensor and the class
    '''
    true_class = self.data[key][1]
    im = Image.open(self.data.imgs[key][0])
    img_tensor = transform(im)

    return img_tensor, true_class

  def __len__(self):
    return len(self.data)

  def get_keys(self):
    '''
      I need this function in the sampler. But keys are just numbers 0:(len-1)
      probably I don't need this function
    '''
    return range(len(self.data))

# just to tensor, i can resize, force to double, normalize, crop, ....
def transform(img):
  t = transforms.Resize([64,64],interpolation=Image.BILINEAR)
  i = t(img)
  i = torch.from_numpy(np.asarray(i))
  i.unsqueeze_(0)
  #i = i.type(torch.cuda.FloatTensor)
  return i

trainingData = customDataset(trainPath,transform)
x = trainingData[1][0]
print(x.shape, x.type())

len(trainingData)

l = trainingData.get_keys()
l[657]

torch.Size([1, 64, 64]) torch.ByteTensor


657

In [71]:
# overwrite sampler class

class customSampler(Sampler):
  '''
    TO DO: 
    - _iter_ which returns an iterable over the dataset
    - _len_ which returns the length of the dataset (needed to compute number of batches in dataloader)
  '''
  def __init__(self, data_source):
    self.data_source = data_source

  def __iter__(self):
    '''
      we don't care the order in which iterate the dataset so this function defines
      an iterator over the key list
    '''
    return iter(self.data_source.get_keys())

  def __len__(self):
    return len(self.data_source.get_keys())  



def split(dataset, val_size):
  '''
    @dataset: a customDataset object
    @val_size: percentage of the dataset that should compose the validation set

    This function allows us to split our dataset into 
    a validation set and a training set. This is used internally in 
    Loader
  '''

  # We want to split our dataset given itself and the % of sample for validation
  num = len(dataset)
  index = list(dataset.get_keys())
  np.random.shuffle(index) # pick at random
  flag_split = int(val_size * num)

  train_index = index[flag_split:]
  validation_index = index[:flag_split]

  # https://pytorch.org/docs/stable/data.html -> Samples elements randomly from a given list of indices, without replacement
  train_sampler = SubsetRandomSampler(train_index)
  validation_sampler = SubsetRandomSampler(validation_index)

  return train_sampler, validation_sampler


def loaders(dataset, val_size, batch_size, num_workers):
  ''' 
    @dataset: a customDataset object
    @val_size: the percentage (in [0,1]) of the validation set data
    @batch_size: the number of data in each batch
    @num_workers: number of subprocesses to use in the data loader
  '''

  train_sampler, validation_sampler = split(dataset, val_size)
  train_loader = DataLoader(dataset,
                            batch_size = batch_size,
                            sampler = train_sampler,
                            num_workers = num_workers)
  val_loader = DataLoader(dataset,
                          batch_size = batch_size,
                          sampler = validation_sampler,
                          num_workers = num_workers)
  return train_loader, val_loader

In [81]:
import torch.nn.functional as F

# now I can try to build a cnn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.input_dim = 1 * 64 * 64
        self.n_classes = 15
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1)
        self.maxpooling = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(12*12*32,15) # 12*12*32 no padding default
        
    def forward(self, x, verbose=False):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        x = self.maxpooling(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        x = self.maxpooling(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = F.relu(x)
        #print(x.shape)
        x = self.fc1(x)
        #print(x.shape)
        x = F.softmax(x)
        
        return x
        
net = CNN()
net.to(device) # so I put the model on GPU
print(net)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (maxpooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=4608, out_features=15, bias=True)
)


In [82]:
# try to apply net just to an image
trainingSet = customDataset(trainPath, transform)
testSet = customDataset(testPath, transform)

im = trainingSet[21][0]
net(im)

torch.Size([1, 64, 64])


RuntimeError: ignored

In [83]:
# train with our classes

# train and test sets
trainingSet = customDataset(trainPath, transform)
testSet = customDataset(testPath, transform)


# train and validation dataloaders
batch_size = 16
num_workers = 1
trainLoader, valLoader = loaders(trainingSet, 0.15, batch_size, num_workers)

lr = 0.001
momentum = 0.9
epochs = 10

n_batches = len(trainLoader)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)

net.train() # set training mode
for e in range(epochs):
    for i, data in enumerate(trainLoader):

        batch = data[0].to(device)
        batch = batch.float()
        labels = data[1].to(device)      
        outputs = net(batch)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            print("[EPOCH]: {}, [BATCH]: {}/{}, [LOSS]: {}".format(e, i, n_batches, loss.item()))
            display.clear_output(wait=True)

# https://stackoverflow.com/questions/59582663/cnn-pytorch-error-input-type-torch-cuda-bytetensor-and-weight-type-torch-cu

torch.Size([16, 1, 64, 64])
torch.Size([16, 8, 62, 62])


RuntimeError: ignored